In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import os
os.chdir('C:\\Users\\nbg05\\Local Documents\\Projects\WhiteHeaven')

import puzzle.lib.puzzletools as tools
from puzzle.lib.puzzlesolving import PuzzlePiece, Puzzle

from ipywidgets import interact
from tqdm.notebook import tqdm

In [2]:
# create a list of all the images in the folder
# Gets the puzzle piece image
def get_piece_image(path):
    # images are of the format piece#.png
    img = cv2.imread(path)
    return img

piece_dir = 'dataset/pieces-clean'
image_files = os.listdir(piece_dir)
image_files = sorted(image_files, key=lambda x: int(x[5:-4]))

# load all off the images
piece_images = []
for i in tqdm(range(len(image_files)), 'images'):
    piece_images.append( get_piece_image(os.path.join(piece_dir, image_files[i])) )

piece_contours = []
for i in tqdm(range(len(piece_images)), 'contours'):
    piece_contours.append(tools.get_contour(piece_images[i], 226))

images:   0%|          | 0/1000 [00:00<?, ?it/s]

contours:   0%|          | 0/1000 [00:00<?, ?it/s]

In [3]:
@interact(i=(0, len(piece_images)-1), zoom=(0, 200), zoom_x=(0, 400), zoom_y=(0, 400))
def view_contour(i, zoom=200, zoom_x=200, zoom_y=200):
    
    start_x = max(0, zoom_x - zoom)
    end_x = min(piece_images[i].shape[1], zoom_x + zoom)
    start_y = max(0, zoom_y - zoom)
    end_y = min(piece_images[i].shape[0], zoom_y + zoom)
    
    fig, ax = plt.subplots()
    fig.set_size_inches(13, 8)
    
    ax.set_xlim(0, end_x-start_x)
    ax.set_ylim(0, end_y-start_y)
    ax.imshow(piece_images[i][start_y:end_y,start_x:end_x])
    ax.plot(piece_contours[i][:,0]-start_x, piece_contours[i][:,1]-start_y, 'r')
    plt.show()

interactive(children=(IntSlider(value=499, description='i', max=999), IntSlider(value=200, description='zoom',…

In [4]:
pieces = []
for i in tqdm(range(len(piece_contours))):
    pieces.append(PuzzlePiece(piece_contours[i]))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
# figure out if any of the pieces are malformed
for i, piece in enumerate(pieces):
    limit = 20
    lengths = [len(edge.points) for edge in piece.edges]
    if abs(lengths[0] - lengths[2]) > limit and abs(lengths[1] - lengths[3]) > limit:
        print(i, lengths)

0 [226, 97, 157, 186]
1 [223, 186, 136, 130]
2 [128, 195, 214, 124]
3 [148, 131, 222, 204]
427 [196, 198, 172, 219]
993 [173, 193, 196, 216]


In [26]:
puzzle = Puzzle(pieces, (40, 25))
puzzle.place_piece(0, 0, 0, 2)

In [7]:
@interact(x=(0, 24), y=(0, 40), piece=(0, len(pieces)-1), j=(0, 3))
def visualize_match(x, y, piece, j):
    edges = puzzle.get_pocket(x, y)
    piece_edges = pieces[piece].edges
    piece_edges = piece_edges[j:] + piece_edges[:j]

    fig, ax = plt.subplots(2, 2)
    fig.set_size_inches(13, 8)
    ax = ax.flatten()

    for i in range(4):
        ax[i].plot(piece_edges[i].normalized[:, 0], piece_edges[i].normalized[:, 1])
        if edges[i].points is None:
            continue
        ax[i].plot(edges[i].normalized[:, 0], edges[i].normalized[:, 1])
    
    plt.show()

def visualize_edge(x, y, piece, i, j):
    edges = puzzle.get_pocket(x, y)
    piece_edges = pieces[piece].edges
    piece_edges = piece_edges[j:] + piece_edges[:j]

    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    fig.set_size_inches(13, 8)

    ax.plot(piece_edges[i].normalized[:, 0], piece_edges[i].normalized[:, 1])
    if not edges[i].points is None:
        ax.plot(edges[i].normalized[:, 0], edges[i].normalized[:, 1])


interactive(children=(IntSlider(value=12, description='x', max=24), IntSlider(value=20, description='y', max=4…

In [28]:
edges = puzzle.get_pocket(0, 1)
piece = pieces[90]
for i in range(4):
    print(i, edges[i].type, piece.edges[i].type)

piece.compare_edges(edges)

0 outer inner
1 None outer
2 None inner
3 flat flat


141.80679758059017

In [34]:
x = 1
y = 1
indices, scores = puzzle.solve_piece(x, y)
print(indices[:10])
print(scores[:10])

[[290   1]
 [692   3]
 [460   1]
 [426   1]
 [925   3]
 [409   3]
 [289   3]
 [180   3]
 [809   3]
 [854   3]]
[336.21421445 387.80843416 392.25460502 416.44479198 418.18257641
 418.91620149 431.59952286 432.79947939 449.11041907 453.32728937]


In [35]:
@interact(i=(0,9))
def visualize(i=0):
    visualize_match(x, y, indices[i, 0], indices[i, 1])

interactive(children=(IntSlider(value=0, description='i', max=9), Output()), _dom_classes=('widget-interact',)…

In [10]:
@interact(i=(0,9), j=(0,3))
def visualize(i=0, j=0):
    visualize_edge(x, y, indices[i, 0], j, indices[i, 1])

interactive(children=(IntSlider(value=0, description='i', max=9), IntSlider(value=0, description='j', max=3), …

In [33]:
i = 0
puzzle.place_piece(x, y, indices[i, 0], indices[i, 1])